# BenSira Collatex

##### Python Packages

In [ ]:
!pip install collatex
!pip install graphviz
!pip install levenshtein
!pip install xmltodict
!pip install beautifulsoup4
!pip install lxml
!pip install textdistance
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install nltk
!pip install spacy==3.2.2
!pip install hebspacy
!pip install openpyxl


### File Parsing

In [1]:
# See if beautiful soup can be of any use
# This part of the code is clunky but it is what it is

# Implementation of the first part
from itertools import combinations
from bs4 import BeautifulSoup
import re

fileNames = ["ms_a_new.xml","ms_b_new.xml","ms_c_new.xml","ms_d_new.xml","ms_e_new.xml","ms_f_new.xml"]
soup = dict()
chapter = dict()
verse_list = dict()
verses = dict()

for currFile in fileNames:
    with open(currFile,encoding="utf8") as fp:
        soup[currFile] = BeautifulSoup(fp,features='xml')
        chs = list()
        verse_list[currFile] = dict()
        verses[currFile] =dict()
        for i in soup[currFile].findAll("chap"):
            ch_string=i.contents[0].strip()
            chs.append(ch_string) # strip to remove trailing spaces or new line characters
            verses_list = list()
            verse = i.findAll("text")
            verses[currFile][ch_string] = dict()
            for verse_iter in verse:
                if verse_iter.verse_nb:
                    verse_num=verse_iter.verse_nb.text.strip()
                verses_list.append(verse_num)
                """
                <!ELEMENT folio (#PCDATA)> <!-- shelfmark of the manuscript and folio number -->
                <!ELEMENT verse_nb (#PCDATA)> <!-- verse (children of chapter) -->
                <!ELEMENT line (#PCDATA)> <!-- line on the manuscript -->
                <!ELEMENT vacat_car (#PCDATA)> <!-- a space into the manuscript -->
                <!ELEMENT greek (#PCDATA)> <!-- greek word or letter -->
                <!ELEMENT reconstructed (#PCDATA)> <!-- Hebrew reconstructed -->
                <!ELEMENT superscript (#PCDATA)> <!-- Hebrew superscript letters or words -->
                <!ELEMENT supralinear (#PCDATA)> <!-- Hebrew supralinear letters or words (I think = superscript) -->
                <!ELEMENT margin_reconstructed (#PCDATA)> <!-- marginal notation reconstructed -->
                <!ELEMENT margin_car (#PCDATA)> <!-- marginal notation -->
                <!ELEMENT margin_infralinear (#PCDATA)> <!-- marginal notation -->
                <!ELEMENT margin_supralinear (#PCDATA)> <!-- marginal notation -->
                """
                # For now clean the text which might be enclosed in the tags
                unwanted_tags= ["folio","verse_nb","line","vacat_car","greek","superscript",\
                "supralinear","margin_reconstructed","margin_car","margin_infralinear",\
                "margin_supralinear","Article"]
                
                # Clean tags
                [s.extract() for s in verse_iter(unwanted_tags)]
                
                # FileName will never be empty
                if (ch_string and verse_num):
                    verses[currFile][ch_string][verse_num] = verse_iter.text.replace("[","").replace("]","")
            verse_list[currFile][ch_string]=verses_list
        
        chapter[currFile] = chs
        #verse = chapter[currFile].findAll("text")
        #for i in verse:
            #print(i.verse_nb.text)
            #print(i.contents)
            #ls = [type(item) for item in i.contents]
            #print(ls)
            #print("\n")


#print(soup['ms_d_new.xml'].findAll("chap")[0].contents[0])



# Implementation of the second part
# Building the combinations
chap_matching = dict()
witnesses = dict()
chap_info = dict()
count = 0
for combo in combinations(fileNames, 2):  # 2 for pairs, 3 for triplets, etc
    matching_chs_list = []
    if combo[0] in chap_matching.keys():
        pass
    else:
        chap_matching[combo[0]] = dict()
    for chap_in_file1 in chapter[combo[0]]:
        for chap_in_file2 in chapter[combo[1]]:
            ch1_num=re.findall(r'\b\d+\b', chap_in_file1)
            ch2_num=re.findall(r'\b\d+\b', chap_in_file2)
            if ch1_num == ch2_num:
                matching_chs_list.append([chap_in_file1,chap_in_file2]) 
                # Lets see if we can find common verses
                common_verses= set(verse_list[combo[0]][chap_in_file1]).intersection(verse_list[combo[1]][chap_in_file2])
                for com_verse in common_verses:
                    if com_verse:
                        witnesses[count] = dict()
                        witnesses[count]['A'] = verses[combo[0]][chap_in_file1][com_verse]
                        witnesses[count]['B'] = verses[combo[1]][chap_in_file2][com_verse]
                        chap_info[count] = (combo[0],chap_in_file1,combo[1],chap_in_file2,com_verse)
                        count = count+1;
    chap_matching[combo[0]][combo[1]]=matching_chs_list
    

# Changing the witness variable here will change the verse which will be loaded for comaparison
witness_index=12
from collatex import *
collation = Collation()
collation.add_plain_witness("A", witnesses[witness_index]['A'])
collation.add_plain_witness("B", witnesses[witness_index]['B'])
print("\nManuscript A: "+witnesses[witness_index]['A'])
print("Manuscript B: "+witnesses[witness_index]['B']+"\n")
#alignment_table = collate(collation, segmentation=False)
#print(chap_matching)
#print(verses)
print(chap_info[witness_index])
alignment_table = collate(collation,output="svg")


            

Error: syntax error in line 1 
... <B></B> ...
in label of node 3



Manuscript A:  באין עצבה אל תאחר	וברב זדיםא‍ל תקומם׃ 
Manuscript B:  ב̇א̇ין עצה אל תתור	ובריב זדים אל תקומם׃

('ms_a_new.xml', 'Chapter 11', 'ms_b_new.xml', 'Siracide 11', '9')


CalledProcessError: Command '[PosixPath('dot'), '-Kdot', '-Tsvg', '-O', 'Digraph.gv']' returned non-zero exit status 1. [stderr: b'Error: syntax error in line 1 \n... <B></B> ...\nin label of node 3\n']

### Import Data into Pandas Dataframe

In [2]:
# We import the chap_info variable into pandas
import pandas as pd
chap_info_df = pd.DataFrame.from_dict(chap_info,orient='index',columns=['Manuscript1','Chapter1','Manuscript2','Chapter2','Verse'])
# Strip ms_a and _new.xml from the manuscript
chap_info_df['Manuscript1'] = chap_info_df['Manuscript1'].str.replace('ms_','')
chap_info_df['Manuscript2'] = chap_info_df['Manuscript2'].str.replace('ms_','')
# Strip _new.xml from the manuscript filenames
chap_info_df['Manuscript1'] = chap_info_df['Manuscript1'].str.replace('_new.xml','')
chap_info_df['Manuscript2'] = chap_info_df['Manuscript2'].str.replace('_new.xml','')
# Add the witnesses to the dataframe
chap_info_df['Witness1'] = chap_info_df.index.map(lambda x: witnesses[x]['A'])
chap_info_df['Witness2'] = chap_info_df.index.map(lambda x: witnesses[x]['B'])


chap_info_df

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...
...,...,...,...,...,...,...,...
148,e,Chapter 33,f,Chapter 33,4,הכין אומר̊ ואחר תעשה׃\tובית מנוח ו̊א̊חר ת̊גיה,הכין אומר ואחר תעשה∙\tובית מנוח ואחר תגיה׃
149,e,Chapter 33,f,Chapter 33,8,אבל ב̶ח̶כ̶מ̶ת̶ בחכמת ייי נשפטו\tויש מהם מועד׃,אבל בחכמת ייי נשפטו∙\tויש מהם מועד
150,e,Chapter 33,f,Chapter 33,5,גלגל קל̊ לב נבל׃\tואופן̊ חוזר מחשבותיו׃,גלגל קל לב נבל∙\tואופן חוזר מחש̇בותי̇ו׃
151,e,Chapter 33,f,Chapter 33,6,כסוס מוכן אוהב שונא׃\tתחת כל אוהב יצהל׃,כסוס מוכן אוהב שונא∙\tתחת כל אוהב יצהל׃


### Clean the witnesses

In [3]:

def clean_hebrew_punctuations(text):
    # Replace tab characters with spaces
    text = text.replace('\t', ' ')
    # Regex pattern for hebrew text excluding punctuation
    pattern = r'[^\u0591-\u05F4\uFB1D-\uFB4F ]|[\u05F3\u05F4\u05BE\u05C0\u05C3\u05C2\u05BD\u059F]'
    # Use re.sub() to replace matched characters with ''
    output_str = re.sub(pattern, '', text)
    return output_str


In [4]:
num_witnesses = len(witnesses)
 # lets clean the witnesses using the clean_hebrew_punctuations function into a new dictionary for collatex
witnesses_clean = dict()
for i in range(num_witnesses):
    witnesses_clean[i] = dict()
    witnesses_clean[i]['A'] = clean_hebrew_punctuations(witnesses[i]['A'])
    witnesses_clean[i]['B'] = clean_hebrew_punctuations(witnesses[i]['B'])

# Apply cleaning function to the dataframe
chap_info_df['Witness1_cleaned'] = chap_info_df['Witness1'].apply(lambda x: clean_hebrew_punctuations(x))
chap_info_df['Witness2_cleaned'] = chap_info_df['Witness2'].apply(lambda x: clean_hebrew_punctuations(x))
chap_info_df

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו
...,...,...,...,...,...,...,...,...,...
148,e,Chapter 33,f,Chapter 33,4,הכין אומר̊ ואחר תעשה׃\tובית מנוח ו̊א̊חר ת̊גיה,הכין אומר ואחר תעשה∙\tובית מנוח ואחר תגיה׃,הכין אומר ואחר תעשה ובית מנוח ואחר תגיה,הכין אומר ואחר תעשה ובית מנוח ואחר תגיה
149,e,Chapter 33,f,Chapter 33,8,אבל ב̶ח̶כ̶מ̶ת̶ בחכמת ייי נשפטו\tויש מהם מועד׃,אבל בחכמת ייי נשפטו∙\tויש מהם מועד,אבל בחכמת בחכמת ייי נשפטו ויש מהם מועד,אבל בחכמת ייי נשפטו ויש מהם מועד
150,e,Chapter 33,f,Chapter 33,5,גלגל קל̊ לב נבל׃\tואופן̊ חוזר מחשבותיו׃,גלגל קל לב נבל∙\tואופן חוזר מחש̇בותי̇ו׃,גלגל קל לב נבל ואופן חוזר מחשבותיו,גלגל קל לב נבל ואופן חוזר מחשבותיו
151,e,Chapter 33,f,Chapter 33,6,כסוס מוכן אוהב שונא׃\tתחת כל אוהב יצהל׃,כסוס מוכן אוהב שונא∙\tתחת כל אוהב יצהל׃,כסוס מוכן אוהב שונא תחת כל אוהב יצהל,כסוס מוכן אוהב שונא תחת כל אוהב יצהל


### Run the actual collation

In [5]:
# Use the collatex library to perform the collation and build the variants

num_witnesses = len(witnesses)
# Lets create an empty dictionary to store the variants
variants = dict()
# For chapter information we can refer to the dictionary chap_info
# The dictionary indexes are same across all the data

# Lets loop of each witness
for witness_ind in range(num_witnesses):
    # Lets run the collation for each set of witnesses
    # Create empty dictionary for each element
    variants[witness_ind] = dict()
    variants[witness_ind]['A'] = '' # Empty initialisatiom
    variants[witness_ind]['B'] = ''
    collation = Collation()
    collation.add_plain_witness("A", witnesses_clean[witness_ind]['A'])
    collation.add_plain_witness("B", witnesses_clean[witness_ind]['B'])
    # Perform the collation
    alignment_table = collate(collation)
    # Lets start trying to find the variants
    for column in alignment_table.columns:
        if column.variant:
        # Add to variants the tokens as strings       
            for manuscript, tokens in column.tokens_per_witness.items():
                token_strings = [token.token_string for token in tokens]
                variants[witness_ind][manuscript] += ' '.join(token_strings)
                variants[witness_ind][manuscript] += ' '


In [6]:
# Variants are now stored in the dataframe
# Insert the variants from the variants variable into the same dataframe
chap_info_df['Variant1'] = chap_info_df.index.map(lambda x: variants[x]['A'])
chap_info_df['Variant2'] = chap_info_df.index.map(lambda x: variants[x]['B'])

chap_info_df.head()

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned,Variant1,Variant2
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים,מושל ושופט,שופט ומושל
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו,בעיניו,נכבד ממנו
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,,בני
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה,נכבד,הנכבד בעיניו בעשרו
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו,יש,איש עשיר


### Implement text distances 

In [7]:
# Apply text distance function to the dataframe between the variants
from textdistance import levenshtein
chap_info_df['Levenshtein'] = chap_info_df.apply(lambda x: levenshtein(x['Variant1'], x['Variant2']), axis=1)
chap_info_df

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned,Variant1,Variant2,Levenshtein
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים,מושל ושופט,שופט ומושל,7
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו,בעיניו,נכבד ממנו,7
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,,בני,3
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה,נכבד,הנכבד בעיניו בעשרו,14
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו,יש,איש עשיר,6
...,...,...,...,...,...,...,...,...,...,...,...,...
148,e,Chapter 33,f,Chapter 33,4,הכין אומר̊ ואחר תעשה׃\tובית מנוח ו̊א̊חר ת̊גיה,הכין אומר ואחר תעשה∙\tובית מנוח ואחר תגיה׃,הכין אומר ואחר תעשה ובית מנוח ואחר תגיה,הכין אומר ואחר תעשה ובית מנוח ואחר תגיה,,,0
149,e,Chapter 33,f,Chapter 33,8,אבל ב̶ח̶כ̶מ̶ת̶ בחכמת ייי נשפטו\tויש מהם מועד׃,אבל בחכמת ייי נשפטו∙\tויש מהם מועד,אבל בחכמת בחכמת ייי נשפטו ויש מהם מועד,אבל בחכמת ייי נשפטו ויש מהם מועד,בחכמת,,6
150,e,Chapter 33,f,Chapter 33,5,גלגל קל̊ לב נבל׃\tואופן̊ חוזר מחשבותיו׃,גלגל קל לב נבל∙\tואופן חוזר מחש̇בותי̇ו׃,גלגל קל לב נבל ואופן חוזר מחשבותיו,גלגל קל לב נבל ואופן חוזר מחשבותיו,,,0
151,e,Chapter 33,f,Chapter 33,6,כסוס מוכן אוהב שונא׃\tתחת כל אוהב יצהל׃,כסוס מוכן אוהב שונא∙\tתחת כל אוהב יצהל׃,כסוס מוכן אוהב שונא תחת כל אוהב יצהל,כסוס מוכן אוהב שונא תחת כל אוהב יצהל,,,0


In [8]:
# Calculate the hamming distance between the variants
from textdistance import hamming
chap_info_df['Hamming'] = chap_info_df.apply(lambda x: hamming(x['Variant1'], x['Variant2']), axis=1)  
chap_info_df.head(10)

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned,Variant1,Variant2,Levenshtein,Hamming
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים,מושל ושופט,שופט ומושל,7,12
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו,בעיניו,נכבד ממנו,7,10
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,,בני,3,4
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה,נכבד,הנכבד בעיניו בעשרו,14,14
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו,יש,איש עשיר,6,6
5,a,Chapter 10,b,Siracide 10,22,גר וזר נכרי ורש\tתפארתם י̊רא̊ת א‍להים•,גר זר נכרי ורש\tתפארתם ירא֟ת ייי׃\t,גר וזר נכרי ורש תפארתם יראת אלהים,גר זר נכרי ורש תפארתם יראת ייי,וזר אלהים,זר ייי,6,11
6,a,Chapter 10,b,Siracide 10,23,אין לבזות ד̇ל̇ מ̊ש̇כיל\tואין לכבד כ̊ל איש חכ̊ם׃,אין לבזות דל משכיל\tואין לכבד כל איש חמס׃,אין לבזות דל משכיל ואין לכבד כל איש חכם,אין לבזות דל משכיל ואין לכבד כל איש חמס,חכם,חמס,3,5
7,a,Chapter 10,b,Siracide 10,25,עבד משכיל הורם\tועבד̊ מש̊כיל ל̊ﬡ יתאונן׃,עבד משכיל חביב כנפש\t\t,עבד משכיל הורם ועבד משכיל לﬡ יתאונן,עבד משכיל חביב כנפש,משכיל הורם ועבד לﬡ יתאונן,חביב כנפש,23,27
8,a,Chapter 10,b,Siracide 10,27,טוב עובד ויותר הון\tממ̊ת̊כבד וח̊סר מתן׃,טוב עובד ויותר הון\tממתכבד̊̊\t,טוב עובד ויותר הון ממתכבד וחסר מתן,טוב עובד ויותר הון ממתכבד,וחסר מתן,,10,10
9,a,Chapter 10,b,Siracide 10,28,בני בענוה כבד נפשך\tויתן לך טעם̊ כ̇י̇ו̇צ̇א̇ בﬣ׃,בני בענוה כבד נפשך\tותן לה טעם כי̊̊ו̊̊צ̊̊א בה׃,בני בענוה כבד נפשך ויתן לך טעם כיוצא בﬣ,בני בענוה כבד נפשך ותן לה טעם כיוצא בה,ויתן לך בﬣ,ותן לה בה,4,12


In [9]:
chap_info_df.head(20)

,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned,Variant1,Variant2,Levenshtein,Hamming
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים,מושל ושופט,שופט ומושל,7,12
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו,בעיניו,נכבד ממנו,7,10
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,,בני,3,4
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה,נכבד,הנכבד בעיניו בעשרו,14,14
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו,יש,איש עשיר,6,6
5,a,Chapter 10,b,Siracide 10,22,גר וזר נכרי ורש\tתפארתם י̊רא̊ת א‍להים•,גר זר נכרי ורש\tתפארתם ירא֟ת ייי׃\t,גר וזר נכרי ורש תפארתם יראת אלהים,גר זר נכרי ורש תפארתם יראת ייי,וזר אלהים,זר ייי,6,11
6,a,Chapter 10,b,Siracide 10,23,אין לבזות ד̇ל̇ מ̊ש̇כיל\tואין לכבד כ̊ל איש חכ̊ם׃,אין לבזות דל משכיל\tואין לכבד כל איש חמס׃,אין לבזות דל משכיל ואין לכבד כל איש חכם,אין לבזות דל משכיל ואין לכבד כל איש חמס,חכם,חמס,3,5
7,a,Chapter 10,b,Siracide 10,25,עבד משכיל הורם\tועבד̊ מש̊כיל ל̊ﬡ יתאונן׃,עבד משכיל חביב כנפש\t\t,עבד משכיל הורם ועבד משכיל לﬡ יתאונן,עבד משכיל חביב כנפש,משכיל הורם ועבד לﬡ יתאונן,חביב כנפש,23,27
8,a,Chapter 10,b,Siracide 10,27,טוב עובד ויותר הון\tממ̊ת̊כבד וח̊סר מתן׃,טוב עובד ויותר הון\tממתכבד̊̊\t,טוב עובד ויותר הון ממתכבד וחסר מתן,טוב עובד ויותר הון ממתכבד,וחסר מתן,,10,10
9,a,Chapter 10,b,Siracide 10,28,בני בענוה כבד נפשך\tויתן לך טעם̊ כ̇י̇ו̇צ̇א̇ בﬣ׃,בני בענוה כבד נפשך\tותן לה טעם כי̊̊ו̊̊צ̊̊א בה׃,בני בענוה כבד נפשך ויתן לך טעם כיוצא בﬣ,בני בענוה כבד נפשך ותן לה טעם כיוצא בה,ויתן לך בﬣ,ותן לה בה,4,12


#### This part of the code tries to assess text inversions

""" In its current form it only labels empty variants as true inversions """
""" This function needs to be looked up """

In [10]:
# Lets detect text inversions
def detect_inversion(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())

    return set1 == set2

# Apply detect inversion function to the dataframe
chap_info_df['Inversion'] = chap_info_df.apply(lambda x: detect_inversion(x['Witness1_cleaned'],x['Witness2_cleaned']),axis=1)
# Count the true and false values in the inversion column
chap_info_df['Inversion'].value_counts()

Inversion
False    119
True      34
Name: count, dtype: int64

ADDITION/DELETION

In [11]:
import difflib

def diff_texts(text1, text2):
    text1_words = text1.split()
    text2_words = text2.split()
    
    d = difflib.Differ()
    diff = d.compare(text1_words, text2_words)
    
    return '\n'.join(diff)

# Apply the diff_texts function to the cleaned witnesses
chap_info_df['Witness_diff'] = chap_info_df.apply(lambda x: diff_texts(x['Witness1_cleaned'],x['Witness2_cleaned']),axis=1)
chap_info_df.head(20)


,Manuscript1,Chapter1,Manuscript2,Chapter2,Verse,Witness1,Witness2,Witness1_cleaned,Witness2_cleaned,Variant1,Variant2,Levenshtein,Hamming,Inversion,Witness_diff
0,a,Chapter 10,b,Siracide 10,24,ש̇ר מושל ושופט נכב̇דו\tו̇א̊ין גד̇ול מ̊ירא א‍ל...,שר שופט ומושל נכבדו\tוא̇ין ג̊̊ד̇ול מירא אלהים\t,שר מושל ושופט נכבדו ואין גדול מירא אלהים,שר שופט ומושל נכבדו ואין גדול מירא אלהים,מושל ושופט,שופט ומושל,7,12,False,שר\n- מושל\n- ושופט\n? -\n\n+ שופט\n+ ומושל\...
1,a,Chapter 10,b,Siracide 10,20,בין אחים ראשם נכבד\tוירא א‍להים בע̇ינ̊יו׃,בין אחים ראשם נכבד\tוירא אלהים נכבד ממנו׃,בין אחים ראשם נכבד וירא אלהים בעיניו,בין אחים ראשם נכבד וירא אלהים נכבד ממנו,בעיניו,נכבד ממנו,7,10,False,בין\n אחים\n ראשם\n נכבד\n וירא\n אלהים...
2,a,Chapter 10,b,Siracide 10,29,מרשיע נפשו מי יצדיקנו\tומי יכבד מקלה נפשו׃,בני מרשיע נפשו מי יצדיקנו\tומי יכבד מ̊̊קלה נפשו׃,מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,בני מרשיע נפשו מי יצדיקנו ומי יכבד מקלה נפשו,,בני,3,4,False,+ בני\n מרשיע\n נפשו\n מי\n יצדיקנו\n ומי...
3,a,Chapter 10,b,Siracide 10,31,נכבד בעשרו איככה\tונקלה בעיניו איככה׃,הנכבד בעיניו ב̊̊ע̇שרו איככה\tונקלה בעשרו בעיני...,נכבד בעשרו איככה ונקלה בעיניו איככה,הנכבד בעיניו בעשרו איככה ונקלה בעשרו בעיניו איככה,נכבד,הנכבד בעיניו בעשרו,14,14,False,- נכבד\n+ הנכבד\n? +\n\n+ בעיניו\n בעשרו\n א...
4,a,Chapter 10,b,Siracide 10,30,יש דל נכבד בגלל שכלו\tויש נכבד בגלל עש̇ר̊ו׃,֟דל נכבד בגלל שכלו\tויש איש עשיר נכבד ב̇ג̇ל̇ל̊...,יש דל נכבד בגלל שכלו ויש נכבד בגלל עשרו,דל נכבד בגלל שכלו ויש איש עשיר נכבד בגלל עשרו,יש,איש עשיר,6,6,False,- יש\n דל\n נכבד\n בגלל\n שכלו\n ויש\n+ א...
5,a,Chapter 10,b,Siracide 10,22,גר וזר נכרי ורש\tתפארתם י̊רא̊ת א‍להים•,גר זר נכרי ורש\tתפארתם ירא֟ת ייי׃\t,גר וזר נכרי ורש תפארתם יראת אלהים,גר זר נכרי ורש תפארתם יראת ייי,וזר אלהים,זר ייי,6,11,False,גר\n- וזר\n? -\n\n+ זר\n נכרי\n ורש\n תפא...
6,a,Chapter 10,b,Siracide 10,23,אין לבזות ד̇ל̇ מ̊ש̇כיל\tואין לכבד כ̊ל איש חכ̊ם׃,אין לבזות דל משכיל\tואין לכבד כל איש חמס׃,אין לבזות דל משכיל ואין לכבד כל איש חכם,אין לבזות דל משכיל ואין לכבד כל איש חמס,חכם,חמס,3,5,False,אין\n לבזות\n דל\n משכיל\n ואין\n לכבד\...
7,a,Chapter 10,b,Siracide 10,25,עבד משכיל הורם\tועבד̊ מש̊כיל ל̊ﬡ יתאונן׃,עבד משכיל חביב כנפש\t\t,עבד משכיל הורם ועבד משכיל לﬡ יתאונן,עבד משכיל חביב כנפש,משכיל הורם ועבד לﬡ יתאונן,חביב כנפש,23,27,False,עבד\n משכיל\n+ חביב\n+ כנפש\n- הורם\n- ועבד...
8,a,Chapter 10,b,Siracide 10,27,טוב עובד ויותר הון\tממ̊ת̊כבד וח̊סר מתן׃,טוב עובד ויותר הון\tממתכבד̊̊\t,טוב עובד ויותר הון ממתכבד וחסר מתן,טוב עובד ויותר הון ממתכבד,וחסר מתן,,10,10,False,טוב\n עובד\n ויותר\n הון\n ממתכבד\n- וחס...
9,a,Chapter 10,b,Siracide 10,28,בני בענוה כבד נפשך\tויתן לך טעם̊ כ̇י̇ו̇צ̇א̇ בﬣ׃,בני בענוה כבד נפשך\tותן לה טעם כי̊̊ו̊̊צ̊̊א בה׃,בני בענוה כבד נפשך ויתן לך טעם כיוצא בﬣ,בני בענוה כבד נפשך ותן לה טעם כיוצא בה,ויתן לך בﬣ,ותן לה בה,4,12,False,בני\n בענוה\n כבד\n נפשך\n- ויתן\n? -\n\...


## END